In [1]:
import pandas as pd
from datetime import datetime, date, timedelta

In [2]:
adls = pd.read_csv("outreachAppointmentAdls.csv")
crfs = pd.read_csv("ClientReviewFormRemovingData.csv")
adls['Date'] = pd.to_datetime(adls['VisitDate']).dt.date
crfs['Date'] = pd.to_datetime(crfs['Date of Review']).dt.date

In [3]:
def breakDown(x,y,monthsBefore, repeatData, noCrfsIncluded, cushion):
    dayLen = monthsBefore * 30
    lastDate = max(x['Date'])
    xs = []
    ys = []
    for i, row in x.iterrows():
        endFrame = row['Date'] + timedelta(days= dayLen)
        if endFrame > lastDate:
            break
        currDf = x[(x['Date'] >= row['Date']) & (x['Date'] <= endFrame)]
        hasHospitilizations = len(y[(y['Date'] <= endFrame + timedelta(days = 30 + cushion)) & (y['Date'] >= endFrame + timedelta(days = 30 - cushion)) & (y['Any Hospitalizations in the last 30 days'] == 'Yes')]) > 0
        xs.append(currDf)
        ys.append(int(hasHospitilizations))
    return xs,ys

In [4]:
## Code to create datasets
## options
monthsBefore = 2 ## number of months used for prediction
repeatData = True ## if patient has a positive in march we use data from feb and march to correspond to
noCrfsIncluded = False ## if the patient has no adls do we treat it as no hospitilizations or omit them
cushion = 7 ## number of days for the end of the frame to result in hospitilization


crfsDf = crfs
adlsDf = adls
crfsID = 'Masked Client ID'
adlsID = 'DeIdentify ID'
crfsDate = 'Date'
adlsDate = 'Date'


IDDfs = {}
xDfs = []
yVals = []
numpys ={}
count = 0
for clientId in adlsDf[adlsID].unique():
    tempx = adlsDf[adlsDf[adlsID] == clientId].sort_values(by = adlsDate)
    tempy = crfsDf[crfsDf[crfsID] == clientId].sort_values(by = crfsDate)
    IDDfs[clientId] = (tempx,tempy)
    x,y = breakDown(tempx,tempy,monthsBefore, repeatData, noCrfsIncluded, cushion)
    xDfs += x
    yVals += y
    count += 1
    print(count / len(adlsDf[adlsID].unique()))

0.006024096385542169
0.012048192771084338
0.018072289156626505
0.024096385542168676
0.030120481927710843
0.03614457831325301
0.04216867469879518
0.04819277108433735
0.05421686746987952
0.060240963855421686
0.06626506024096386
0.07228915662650602
0.0783132530120482
0.08433734939759036
0.09036144578313253
0.0963855421686747
0.10240963855421686
0.10843373493975904
0.1144578313253012
0.12048192771084337
0.12650602409638553
0.13253012048192772
0.13855421686746988
0.14457831325301204
0.15060240963855423
0.1566265060240964
0.16265060240963855
0.1686746987951807
0.1746987951807229
0.18072289156626506
0.18674698795180722
0.1927710843373494
0.19879518072289157
0.20481927710843373
0.21084337349397592
0.21686746987951808
0.22289156626506024
0.2289156626506024
0.23493975903614459
0.24096385542168675
0.2469879518072289
0.25301204819277107
0.25903614457831325
0.26506024096385544
0.2710843373493976
0.27710843373493976
0.28313253012048195
0.2891566265060241
0.29518072289156627
0.30120481927710846
0.307

In [7]:
xDfs[0]

,DeIdentify ID,CaregiverID,VisitDate,ActualTimeIn,ActualTimeOut,ProviderID_127318.0,ProviderID_486278.0,ProviderID_203236159.0,ServiceTypeName_AC Agency with Choice,ServiceTypeName_AC Attendant Care,...,TaskCodeGroupName_Toileting:Bathroom (58),TaskCodeGroupName_Toileting:Urinal (60),TaskCodeGroupName_Transfer:Chair (61),TaskCodeGroupName_Transfer:Gait Belt (62),TaskCodeGroupName_Transfer:Hoyer (63),TaskCodeGroupName_Transfer:Transfer (65),TaskCodeGroupName_Transfer:Walker (66),TaskCodeGroupName_Transfer:Wheelchair (67),TaskCodeGroupName_Transportation:Client Errands - Do Not Transport Client (68),Date
0,Client001,Caregiver001,2021-06-01 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-01
1,Client001,Caregiver001,2021-06-02 00:00:00,1900-01-01 10:00:00,1900-01-01 12:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-02
2,Client001,Caregiver001,2021-06-03 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-03
3,Client001,Caregiver001,2021-06-07 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-07
4,Client001,Caregiver001,2021-06-08 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-08
5,Client001,Caregiver001,2021-06-09 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-09
6,Client001,Caregiver001,2021-06-10 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-10
7,Client001,Caregiver001,2021-06-14 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-14
8,Client001,Caregiver001,2021-06-15 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-15
9,Client001,Caregiver001,2021-06-17 00:00:00,1900-01-01 10:00:00,1900-01-01 13:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,2021-06-17


In [41]:
freqDf = pd.DataFrame()
for i in range(len(xDfs)):
    row = xDfs[i].iloc[:,5:78].sum(axis =  0)
    row = row.append(pd.Series(yVals[i], index = ['hasHospitilization']))
    freqDf = freqDf.append(row, ignore_index = True)

In [43]:
freqDf.to_csv("frequencies.csv", index = True)
## Add one month of buffer between interval and crf done 
## build df of frequencies 1 row per df in xDfs done

179.0